In [1]:
import pandas as pd
import requests 
import matplotlib.pyplot as plt
import seaborn
import os

In [2]:
dates = {'begin': '2023-03-01', 'end': '2023-09-01'}
visits_request = requests.get('https://data-charts-api.hexlet.app/visits', params=dates)
visits_data = visits_request.json()
visits_df = pd.DataFrame(visits_data)

In [3]:
registrations_request = requests.get('https://data-charts-api.hexlet.app/registrations', params=dates)
registrations_data = registrations_request.json()
registrations_df = pd.DataFrame(registrations_data)

In [4]:
visits_df = visits_df[~visits_df['user_agent'].str.contains('bot')]
visits_unique = pd.DataFrame(visits_df.groupby('visit_id').agg('max')['datetime']).reset_index()

In [5]:
unique_visits = pd.merge(visits_df, visits_unique, on=['visit_id', 'datetime'])
unique_visits['date'] = pd.to_datetime(unique_visits['datetime']).dt.date

In [6]:
grouped_unique_visits = unique_visits.drop(['user_agent', 'datetime'], axis=1).groupby(['date', 'platform']).agg('count')
grouped_unique_visits.columns = ['visits']

In [7]:
registrations_df['date'] = pd.to_datetime(registrations_df['datetime']).dt.date
grouped_registrations = registrations_df.drop(['registration_type', 'email', 'datetime'], axis=1).groupby(['date', 'platform']).agg('count')
grouped_registrations.columns = ['registrations']

In [8]:
conversions = pd.merge(grouped_unique_visits, grouped_registrations, on=['date', 'platform'])
conversions['conversion'] = (conversions['registrations'] / conversions['visits'] * 100 )
conversions = conversions.reset_index()
conversions.columns = ['date_group', 'platform', 'visits', 'registrations', 'conversion']

In [9]:
conversions = conversions.sort_values('date_group')
conversions.to_json('conversion.json')

Шаг 4

In [10]:
ads_df=pd.read_csv('ads.csv')
ads_df['date_group'] = pd.to_datetime(ads_df['date']).dt.date
ads_df = ads_df.drop(['utm_source', 'utm_medium', 'date'], axis=1)

In [11]:
conversions_dropped = conversions.drop(['platform', 'conversion'], axis=1)
conversions_dropped = conversions_dropped.groupby('date_group').agg('sum')

In [12]:
conv_ads = pd.merge(conversions_dropped, ads_df,  on=['date_group'], how='left')
conv_ads = conv_ads[['date_group', 'visits', 'registrations', 'cost', 'utm_campaign']]

In [13]:
conv_ads['cost'] = conv_ads['cost'].fillna(0)
conv_ads['utm_campaign'] = conv_ads['utm_campaign'].fillna('none')

In [14]:
conv_ads = conv_ads.sort_values('date_group')
conv_ads.to_json('ads.json')

Шаг 5

In [16]:
os.mkdir('charts')

FileExistsError: [WinError 183] Невозможно создать файл, так как он уже существует: 'charts'

1. Итоговые визиты

In [ ]:
x = conv_ads['date_group']
y = conv_ads['visits']
plt.figure(figsize=(20, 5))
bars = plt.bar(x, y, color='#e66a56', label='Итоговые визиты')
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f'{height:.0f}', ha='center', va='bottom')
#plt.legend()
plt.grid(axis='y', linewidth=0.5, alpha=0.5)
plt.xlabel('Дата')
plt.xlim([conv_ads['date_group'].min() - pd.Timedelta(days=2), conv_ads['date_group'].max() + pd.Timedelta(days=2)])
plt.xticks(x[::3], x[::3], rotation=45, fontsize=10)
plt.ylabel('Суммарное количество посещений сайта')
plt.title('Количество посещений сайта в отчётном периоде')
plt.tight_layout()
plt.savefig('./charts/visits.png')

2. Итоговые визиты с разбивкой по платформам: web, android, ios

In [ ]:
grouped_unique_visits = grouped_unique_visits.reset_index()

In [ ]:
x = conv_ads['date_group']
android = grouped_unique_visits[grouped_unique_visits['platform'] == 'android']
web = grouped_unique_visits[grouped_unique_visits['platform'] == 'web']
ios = grouped_unique_visits[grouped_unique_visits['platform'] == 'ios']
plt.figure(figsize=(20, 10))
plt.bar(x, web['visits'], color='#acb78e', label='Web')
plt.bar(x, android['visits'], color='#aae11e', label='Android')
plt.bar(x, ios['visits'], color='#31adc1', label='Ios')
plt.legend(loc='upper right')
plt.grid(linewidth=0.5, alpha=0.5)
plt.xlabel('Дата')
plt.xlim([conv_ads['date_group'].min(), conv_ads['date_group'].max()])
plt.xticks(x[::3], x[::3], rotation=45, fontsize=10)
plt.ylabel('Количество посещений сайта')
plt.title('Количество посещений сайта в отчётном периоде c разбивокй по платформам')
plt.tight_layout()
plt.savefig('./charts/visits_per_platform.png')

3. Итоговые регистрации

In [ ]:
x = conv_ads['date_group']
y = conv_ads['registrations']
plt.figure(figsize=(20, 5))
bars = plt.bar(x, y, color='#ab4bb5', label='Итоговые визиты')
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f'{height:.0f}', ha='center', va='bottom')
#plt.legend()
plt.grid(axis='y', linewidth=0.5, alpha=0.5)
plt.xlabel('Дата')
plt.xlim([conv_ads['date_group'].min() - pd.Timedelta(days=2), conv_ads['date_group'].max() + pd.Timedelta(days=2)])
plt.xticks(x[::3], x[::3], rotation=45, fontsize=10)
plt.ylabel('Суммарное количество регистраций')
plt.title('Количество регистраций новых клиентов в отчётном периоде')
plt.tight_layout()
plt.savefig('./charts/registrations.png')

4. Итоговые регистрации с разбивкой по платформе: web, android, ios

In [ ]:
grouped_registrations = grouped_registrations.reset_index()

In [ ]:
x = conv_ads['date_group']
android = grouped_registrations[grouped_registrations['platform'] == 'android']
web = grouped_registrations[grouped_registrations['platform'] == 'web']
ios = grouped_registrations[grouped_registrations['platform'] == 'ios']
plt.figure(figsize=(20, 10))
plt.bar(x, web['registrations'], color='#acb78e', label='Web')
plt.bar(x, android['registrations'], color='#aae11e', label='Android')
plt.bar(x, ios['registrations'], color='#31adc1', label='Ios')
plt.legend(loc='upper right')
plt.grid(linewidth=0.5, alpha=0.5)
plt.xlabel('Дата')
plt.xlim([conv_ads['date_group'].min(), conv_ads['date_group'].max()])
plt.xticks(x[::3], x[::3], rotation=45, fontsize=10)
plt.ylabel('Количество регистраций')
plt.title('Количество регистраций новых клиентов в отчётном периоде c разбивокй по платформам')
plt.tight_layout()
plt.savefig('./charts/registrations_per_platform.png')

5. Конверсия по каждой платформе

In [ ]:
x = conv_ads['date_group']
android_conv = conversions[conversions['platform'] == 'android']
android_conv = android_conv['conversion'].reset_index(drop=True)
web_conv = conversions[conversions['platform'] == 'web']
web_conv = web_conv['conversion'].reset_index(drop=True)
ios_conv = conversions[conversions['platform'] == 'ios']
ios_conv = ios_conv['conversion'].reset_index(drop=True)

plt.figure(figsize=(20, 15), constrained_layout=True)

plt.subplot(3, 1, 1)
plot = plt.plot(x, android_conv, color='#aae11e', label='Android', marker='o')
for dot, dot_val in enumerate(android_conv):
    plt.text(x[dot], android_conv[dot], f'{dot_val:.0f}%')
plt.grid(axis='both', linewidth=0.5, alpha=0.5)
plt.xlabel('Дата', fontsize = 12)
plt.xlim([conv_ads['date_group'].min() - pd.Timedelta(days=1), conv_ads['date_group'].max() + pd.Timedelta(days=3)])
plt.xticks(x[::5], x[::5], rotation=45, fontsize=10)
plt.ylabel('Конверсия, %', fontsize = 12)
plt.title('Конверсия Android', fontsize = 14)

plt.subplot(3, 1, 2)
plot = plt.plot(x, web_conv, color='#acb78e', label='Web', marker='o')
for dot, dot_val in enumerate(web_conv):
    plt.text(x[dot], web_conv[dot], f'{dot_val:.0f}%')
plt.grid(axis='both', linewidth=0.5, alpha=0.5)
plt.xlabel('Дата', fontsize = 12)
plt.xlim([conv_ads['date_group'].min() - pd.Timedelta(days=1), conv_ads['date_group'].max() + pd.Timedelta(days=3)])
plt.xticks(x[::5], x[::5], rotation=45, fontsize=10)
plt.ylabel('Конверсия, %', fontsize = 12)
plt.title('Конверсия Web', fontsize = 14)

plt.subplot(3, 1, 3)
plot = plt.plot(x, ios_conv, color='#31adc1', label='Итоговые визиты', marker='o')
for dot, dot_val in enumerate(ios_conv):
    plt.text(x[dot], ios_conv[dot], f'{dot_val:.0f}%')
plt.grid(axis='both', linewidth=0.5, alpha=0.5)
plt.xlabel('Дата', fontsize = 12)
plt.xlim([conv_ads['date_group'].min() - pd.Timedelta(days=1), conv_ads['date_group'].max() + pd.Timedelta(days=3)])
plt.xticks(x[::5], x[::5], rotation=45, fontsize=10)
plt.ylabel('Конверсия, %', fontsize = 12)
plt.title('Конверсия Ios', fontsize = 14)

plt.savefig('./charts/conversion_per_platform.png')

6. Средняя конверсия

In [ ]:
conv_grouped = conv_ads[['date_group', 'visits', 'registrations']]
conv_grouped = conv_grouped.copy()
conv_grouped.loc[:, 'conversion_overall'] = conv_grouped['registrations'] / conv_grouped['visits'] * 100

In [ ]:
x = conv_grouped['date_group']
y = conv_grouped['conversion_overall']
plt.figure(figsize=(20, 5))
plot = plt.plot(x, y, color='#e66a56', label='Общая конверсия', marker='o')
for dot, dot_val in enumerate(y):
    plt.text(x[dot], y[dot] + 0.5, f'{dot_val:.0f}%')
plt.legend()
plt.grid(axis='y', linewidth=0.5, alpha=0.5)
plt.xlabel('Дата')
plt.xlim([conv_ads['date_group'].min(), conv_ads['date_group'].max()])
plt.xticks(x[::7], x[::7], rotation=45, fontsize=10)
plt.ylabel('Конверсия, %')
plt.title('Общая конверсия в отчётном периоде')
plt.tight_layout()
plt.savefig('./charts/conversion_overall.png')

7. Стоимости реклам

In [ ]:
x = conv_ads['date_group']
y = conv_ads['cost']
plt.figure(figsize=(20, 5))
plot = plt.plot(x, y, color='#cd4e00', label='Общая стоимость рекламы', marker='o')
for dot, dot_val in enumerate(y):
    plt.text(x[dot], y[dot] + 0.5, f'{dot_val:.0f} руб.')
#plt.legend()
plt.grid(axis='y', linewidth=0.5, alpha=0.5)
plt.xlabel('Дата')
plt.xlim([conv_ads['date_group'].min() - pd.Timedelta(days=1), conv_ads['date_group'].max() + pd.Timedelta(days=3)])
plt.xticks(x[::7], x[::7], rotation=45, fontsize=10)
plt.ylabel('Стоимость, руб.')
plt.title('Общая стоимость рекламы')
plt.tight_layout()
plt.savefig('./charts/campaign_cost_overall.png')

8. Визиты за весь период с цветовым выделением рекламной кампании

In [ ]:
campaigns_period = conv_ads[conv_ads['utm_campaign'] != 'none']
campaign_date = campaigns_period.groupby('utm_campaign')['date_group'].agg(['min', 'max']).reset_index()
campaign_date['color'] = ['#8dbf40', '#cd5b45', '#32a4ba', '#f3ea16','#ababab']
campaign_date.columns = ['campaign', 'begin', 'end', 'color']

In [ ]:
x = conv_ads['date_group']
y = conv_ads['visits']
plt.figure(figsize=(20, 7))
bars = plt.plot(x, y, color='#e66a56', label='Посещения', marker='o')
plt.grid(axis='both', linewidth=0.5, alpha=0.5)
plt.xlabel('Дата')
plt.xlim([conv_ads['date_group'].min() - pd.Timedelta(days=1), conv_ads['date_group'].max() + pd.Timedelta(days=1)])
plt.xticks(x[::7], x[::7], rotation=45, fontsize=10)
plt.ylabel('Суммарное количество посещений сайта')
plt.title('Количество посещений сайта')
plt.axhline(y.mean(), color='gray', linestyle='--', label='Среднее значение', alpha=0.5)
for campaign_number in range(len(campaign_date)):
    plt.axvspan(campaign_date['begin'].iloc[campaign_number], campaign_date['end'].iloc[campaign_number], color=campaign_date['color'].iloc[campaign_number], alpha=0.3, label=campaign_date['campaign'].iloc[campaign_number])
plt.legend(loc='upper right')
plt.tight_layout()
plt.savefig('./charts/visits_with_campaigns.png')

9. Регистрации за весь период с цветовым выделением рекламной кампании

In [ ]:
x = conv_ads['date_group']
y = conv_ads['registrations']
plt.figure(figsize=(20, 7))
bars = plt.plot(x, y, color='#ab4bb5', label='Регистрации', marker='o')
plt.grid(axis='y', linewidth=0.5, alpha=0.5)
plt.xlabel('Дата')
plt.xlim([conv_ads['date_group'].min(), conv_ads['date_group'].max()])
plt.xticks(x[::7], x[::7], rotation=45, fontsize=10)
plt.ylabel('Суммарное количество регистраций')
plt.title('Количество регистраций новых клиентов')
plt.axhline(y.mean(), color='red', linestyle='--', label='Среднее значение', alpha=0.5)
for campaign_number in range(len(campaign_date)):
    plt.axvspan(campaign_date['begin'].iloc[campaign_number], campaign_date['end'].iloc[campaign_number], color=campaign_date['color'].iloc[campaign_number], alpha=0.3, label=campaign_date['campaign'].iloc[campaign_number])
plt.legend(loc='upper right')
plt.tight_layout()
plt.savefig('./charts/registrations_with_campaigns.png')